- Прочитайте главы 4-6 из книги "Spark: The Definitive Guide".
- Загрузите датасет из предыдущей лабораторной работы:
    - https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2022
- Выполните задания.

In [2]:
# Создаем SparkSession
import mappings as mappings
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]")\
    .appName('SparkLab2')\
    .getOrCreate()

# Загружаем датасет.
tiktokData0 = spark\
    .read\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .csv("/home/student/Work/Labs/TikTok_songs_2022.csv")

22/10/28 12:29:47 WARN Utils: Your hostname, student-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/10/28 12:29:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/28 12:29:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


№1: В столбце "loudness" переведите значения из дБ в проценты громкости. Выведите топ10 самых громких песен.

|          track_name|               album|        artist_name|loudness|
|--------------------|--------------------|-------------------|--------|
|          Astronomia|          Astronomia|           Vicetone|      55|
|     Sweater Weather|         I Love You.|  The Neighbourhood|      52|
|          Dandelions|          Safe Haven|            Ruth B.|      51|
|1, 2, 3 (feat. Ja...|1, 2, 3 (feat. Ja...|        Sofía Reyes|      49|
|   Beauty And A Beat|   Beauty And A Beat|              Other|      49|
|Being Good Is Boring|Being Good Is Boring|              Other|      48|
|Wellerman - Sea S...|Wellerman (Sea Sh...|       Nathan Evans|      47|
|           Thot Shit|           Thot Shit|Megan Thee Stallion|      45|
|Friday (feat. Muf...|Friday (feat. Muf...|              Riton|      45|
|               Hawái|        PAPI JUANCHO|             Maluma|      45|

In [44]:
from math import log10
from pyspark.sql.functions import desc
from pyspark.sql.functions import col
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType


tiktokData1 = tiktokData0.withColumn("loudness", (100*10)**(col("loudness")/10))\

tiktokData1.sort(desc("loudness")).select("track_name", "album", "artist_name", "loudness").show(10)

+--------------------+--------------------+-------------------+-------------------+
|          track_name|               album|        artist_name|           loudness|
+--------------------+--------------------+-------------------+-------------------+
|          Astronomia|          Astronomia|           Vicetone| 0.1621063398155203|
|     Sweater Weather|         I Love You.|  The Neighbourhood|0.14354894333536555|
|          Dandelions|          Safe Haven|            Ruth B.|0.12959850747551394|
|1, 2, 3 (feat. Ja...|1, 2, 3 (feat. Ja...|        Sofía Reyes|0.11651987230250002|
|   Beauty And A Beat|   Beauty And A Beat|             Glamii|0.11643941101974166|
|Being Good Is Boring|Being Good Is Boring|          Jena Rose|0.11033162127669813|
|Wellerman - Sea S...|Wellerman (Sea Sh...|       Nathan Evans|0.10556015032159294|
|           Thot Shit|           Thot Shit|Megan Thee Stallion|0.09392906516183994|
|Friday (feat. Muf...|Friday (feat. Muf...|              Riton|0.09392906516

№2: Получите имена соисполнителей из названий песен. Вынесите их в отдельный столбец. Удалите информацию о соисполнителях из названий треков.

|track_name                                     |artist_name      |feat                       |
|-----------------------------------------------|-----------------|---------------------------|
|INDUSTRY BABY                                  |Lil Nas X        |Jack Harlow                |
|Left and Right (Charlie Puth) - Sped Up Version|sped up nightcore|Jung Kook of BTS           |
|Bam Bam                                        |Camila Cabello   |Ed Sheeran                 |
|Down Under                                     |Luude            |Colin Hay                  |
|1, 2, 3                                        |Sofía Reyes      |Jason Derulo & De La Ghetto|
|Chaa Chaa Chaa                                 |Girll Codee      |HoodCelebrityy             |

<div style="text-align: center"> only showing top 6 rows </div>

In [45]:
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import length

tiktokData2 = tiktokData1\
    .withColumn('feat', regexp_extract(col('track_name'), '(.*)\((feat\\. |with )(.+)\)', 3))\
    .withColumn('track_name', regexp_replace('track_name', '\((feat\\. |with )(.+)\)', ''))

tiktokData2.filter(length(col('feat')) > 1).select("track_name", "artist_name", "feat").show(20, truncate=False)

+-------------------------------------------------+---------------+---------------------------+
|track_name                                       |artist_name    |feat                       |
+-------------------------------------------------+---------------+---------------------------+
|STAY                                             |The Kid LAROI  |Justin Bieber              |
|Enemy  - from the series Arcane League of Legends|Imagine Dragons|JID                        |
|Flex On My Ex                                    |KillBunk       |Bankrol Hayden             |
|INDUSTRY BABY                                    |Lil Nas X      |Jack Harlow                |
|Cooped Up                                        |Post Malone    |Roddy Ricch                |
|Bam Bam                                          |Camila Cabello |Ed Sheeran                 |
|Down Under                                       |Luude          |Colin Hay                  |
|1, 2, 3                                

№3: Разделите строки с соисполнителями по символу '&'. Для каждой песни соберите всех исполнителей в один массив. Отсортируйте песни по количеству исполнителей и названиям песен.

|track_name                       |artist_names                             |
|---------------------------------|-----------------------------------------|
|1, 2, 3                          |[Sofía Reyes, Jason Derulo, De La Ghetto]|
|For The Night                    |[Pop Smoke, Lil Baby, DaBaby]            |
|Friday - Dopamine Re-Edit        |[Riton, Mufasa, Hypeman]                 |
|Peaches                          |[Justin Bieber, Daniel Caesar, Giveon]   |
|Bam Bam                          |[Camila Cabello, Ed Sheeran]             |
|Banana - DJ FLe - Minisiren Remix|[Conkarah, Shaggy]                       |

<div style="text-align: center"> only showing top 6 rows </div>

In [46]:
import pyspark.sql.functions as F
from pyspark.sql.functions import split
from pyspark.sql.functions import array_union
from pyspark.sql.functions import array
from pyspark.sql.functions import size
from pyspark.sql.functions import when
from pyspark.sql.functions import udf

tiktokData3 = tiktokData2.withColumn('artist_names',
                                     array_union(split(col('artist_name'), ' & '), array(col('feat'))))
tiktokData3 = tiktokData3.withColumn('artist_names',
                                     F.expr('filter(artist_names, x -> x is not null and x != "")'))

tt4 = tiktokData3

tiktokData3.filter(size(col("artist_names")) > 1).select("track_name", "artist_names").show(20, truncate=False)

+-------------------------------------------------+------------------------------------------+
|track_name                                       |artist_names                              |
+-------------------------------------------------+------------------------------------------+
|STAY                                             |[The Kid LAROI, Justin Bieber]            |
|Enemy  - from the series Arcane League of Legends|[Imagine Dragons, JID]                    |
|Flex On My Ex                                    |[KillBunk, Bankrol Hayden]                |
|INDUSTRY BABY                                    |[Lil Nas X, Jack Harlow]                  |
|Cooped Up                                        |[Post Malone, Roddy Ricch]                |
|Bam Bam                                          |[Camila Cabello, Ed Sheeran]              |
|Down Under                                       |[Luude, Colin Hay]                        |
|1, 2, 3                                          

№4: Выведите список песен Doja Cat.

|  track_name|               album|        artist_names|
|------------|--------------------|--------------------|
|       Woman|          Planet Her|          [Doja Cat]|
|Kiss Me More|Kiss Me More (fea...|     [Doja Cat, SZA]|
|Need to Know|          Planet Her|          [Doja Cat]|
|  Ain't Shit|          Planet Her|          [Doja Cat]|
|   You Right|          Planet Her|          [Doja Cat]|
| Best Friend|Best Friend (feat...|[Saweetie, Doja Cat]|
|       Freak|               Freak|          [Doja Cat]|
|  Boss Bitch|          Boss Bitch|          [Doja Cat]|
|      Say So|            Hot Pink|          [Doja Cat]|

In [47]:
from pyspark.sql.functions import explode, array_contains

tiktokData3.select("track_name", "album", "artist_names").where(array_contains(col('artist_names'), 'Doja Cat')).show(
    truncate=False)

+-------------+---------------------------------------+-------------------------+
|track_name   |album                                  |artist_names             |
+-------------+---------------------------------------+-------------------------+
|Woman        |Planet Her                             |[Doja Cat]               |
|Kiss Me More |Kiss Me More (feat. SZA)               |[Doja Cat, SZA]          |
|Need to Know |Planet Her                             |[Doja Cat]               |
|Ain't Shit   |Planet Her                             |[Doja Cat]               |
|You Right    |Planet Her                             |[Doja Cat]               |
|Best Friend  |Best Friend (feat. Doja Cat) [Remix EP]|[Saweetie, Doja Cat]     |
|motive       |Positions (Deluxe)                     |[Ariana Grande, Doja Cat]|
|Freak        |Freak                                  |[Doja Cat]               |
|Boss Bitch   |Boss Bitch                             |[Doja Cat]               |
|Say So       |H

№5: Выведите таблицу с исполнителями и количеством их треков в порядке уменьшения.

|   artist_name|count|
|--------------|-----|
|      Doja Cat|    9|
|        Coopex|    6|
|     Dame Dame|    5|
|Alex Alexander|    4|
|       YES YES|    4|
|     Lil Nas X|    4|


<div style="text-align: center"> only showing top 6 rows </div>

In [48]:
tiktokData3.groupby(col('artist_name')).count().orderBy('count', ascending=False).show(6)

+--------------+-----+
|   artist_name|count|
+--------------+-----+
|      Doja Cat|    8|
|        Coopex|    6|
|     Dame Dame|    5|
|Alex Alexander|    4|
|       YES YES|    4|
|     Lil Nas X|    4|
+--------------+-----+
only showing top 6 rows



№6: Замените имена всех исполнителей, популярность которых неизвестна или меньше 50, на "Other". Отсортируйте треки по количеству исполнителей и популярности.

|track_name               |artist_names                   |track_pop|
|-------------------------|-------------------------------|---------|
|Peaches                  |[Justin Bieber, Other, Other]  |86       |
|Friday - Dopamine Re-Edit|[Riton, Other, Other]          |83       |
|1, 2, 3                  |[Sofía Reyes, Other, Other]    |82       |
|For The Night            |[Pop Smoke, Other, DaBaby]     |80       |
|Jimmy Cooks              |[Drake, Other]                 |92       |
|INDUSTRY BABY            |[Lil Nas X, Jack Harlow]       |86       |
|Levitating               |[Dua Lipa, DaBaby]             |85       |
|Bam Bam                  |[Camila Cabello, Ed Sheeran]   |83       |
|Beautiful Mistakes       |[Maroon 5, Megan Thee Stallion]|82       |
|Kiss Me More             |[Doja Cat, SZA]                |82       |

<div style="text-align: center"> only showing top 10 rows </div>

In [49]:
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import collect_list, avg


change_names_udf = udf(lambda arr: ["Other" if tiktokData3.filter(col("artist_name") == x).agg(avg(col("artist_pop"))) < 50 else x for x in arr],
                   ArrayType(StringType()))

res = tiktokData3.withColumn('replaced_names', change_names_udf(col('artist_names')))

res.sort(desc('track_pop')).select("track_name", "replaced_names", "track_pop").show(10)

Traceback (most recent call last):
  File "/home/student/Work/lib/python3.10/site-packages/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/home/student/Work/lib/python3.10/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/home/student/Work/lib/python3.10/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
  File "/home/student/Work/lib/python3.10/site-packages/pyspark/context.py", line 447, in __getnewargs__
    raise RuntimeError(
RuntimeError: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.


PicklingError: Could not serialize object: RuntimeError: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.